In [1]:
import os 
import speech_recognition as sr 
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer 
from flask import Flask, request, jsonify
from flask_cors import CORS
from pydub import AudioSegment, silence 
from transformers import pipeline
import spacy
import moviepy.editor as mp 
from moviepy.editor import VideoFileClip  
from googletrans import Translator
from gensim.summarization import summarize
from werkzeug.utils import secure_filename

c:\Users\haris\miniconda3\envs\forPyTorch\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
app = Flask(__name__)
CORS(app, origins="http://localhost:3000", supports_credentials=True)

nlp = spacy.load("en_core_web_sm")

@app.route('/api/analyze-audio/<string:text>', methods=['GET'])
def analyze_text(text):
    if len(text) == 0:
        return jsonify({'error': 'No text provided'}), 400
    
    doc = nlp(text) 
    person_count = len([ent.text for ent in doc.ents if ent.label_ == "PERSON"]) 
    topics = [token.text for token in doc if token.is_alpha and not token.is_stop]
    
    return jsonify({'person_count' : person_count + 1 , 'topic' : topics[0], 'topic2' : topics[1]})
 

@app.route('/api/convert-video-to-mp3', methods=['POST'])
def convert_video_to_mp3():
    if 'file' not in request.files:
        return jsonify({'error': 'No file provided'}), 400

    video_file = request.files['file']
    if video_file.filename == '':
        return jsonify({'error': 'No selected file'}), 400

    if video_file: 
        try: 
            video_path = os.path.join('./uploads', video_file.filename)   
            video_file.save(video_path)
 
            audio_path = os.path.abspath('./uploads/output.mp3') 
            video_clip = VideoFileClip(video_path)
            audio_clip = video_clip.audio
            audio_clip.write_audiofile(audio_path)
            audio_clip.close()
            video_clip.close()

            return jsonify({'message': 'Video converted to MP3 successfully', 'audio_file': audio_path}), 200
        except Exception as e:
            return jsonify({'error': 'Conversion error', 'details': str(e)}), 500


def get_audio_length(audio_filename):
    audio = AudioSegment.from_file(audio_filename)
    return len(audio) / 1000  # Return length in seconds    

def split_audio_with_timestamps(input_audio, output_dir, segment_length_ms=60000):
    print("Processing input audio...")
    audio = AudioSegment.from_file(input_audio)
    audio_length = len(audio)
    silence_ranges = silence.detect_nonsilent(audio, min_silence_len=100, silence_thresh=-40)
    
    segment_texts = []
    segment_start_times = []
    segment_end_times = []
    
    for i, (start, end) in enumerate(silence_ranges):
        segment_start_times.append(start)
        segment_end_times.append(end)
        segment = audio[start:end]
        segment.export(os.path.join(output_dir, f"segment_{i}.wav"), format="wav")
        
    if len(segment_start_times) > 0:
        segment_start_times.insert(0, 0)
        segment_end_times.append(audio_length)
        for i in range(len(segment_start_times) - 1):
            start_time = segment_start_times[i] / 1000
            end_time = segment_end_times[i] / 1000
            segment_texts.append(f"[{start_time}:{end_time}] " + transcribe_audio_segment(os.path.join(output_dir, f"segment_{i}.wav")))
    
    return segment_texts

def transcribe_audio_segment(audio_file):
    recognizer = sr.Recognizer()
    with sr.AudioFile(audio_file) as source:
        audio_text = recognizer.record(source)
        text = recognizer.recognize_google(audio_text)
    return text

def join_transcribed_texts(texts):
    return " ".join(texts)

def convert_mp3_to_wav(mp3_filename):
    wav_filename = mp3_filename.replace('.mp3', '.wav')
    audio = AudioSegment.from_mp3(mp3_filename)
    audio.export(wav_filename, format="wav")
    return wav_filename

@app.route('/api/convert-mp3-to-text', methods=['POST'])
def convert_mp3_to_text():
    if 'file' not in request.files:
        return jsonify({'error': 'No file part'}), 400

    file = request.files['file']
    language = request.form['language']
    
    if file.filename == '':
        return jsonify({'error': 'No selected file'}), 400

    if file:  
        mp3_filename = os.path.join('uploads', file.filename)
        file.save(mp3_filename)
        segment_dir = 'segments'  
        os.makedirs(segment_dir, exist_ok=True)
        audio_length = get_audio_length(mp3_filename)

        if audio_length > 60:
            split_audio_with_timestamps(mp3_filename, segment_dir, segment_length_ms=60000)
            transcribed_texts = transcribe_audio_segment(segment_dir) 
            joined_text = join_transcribed_texts(transcribed_texts)
        else:
            recognizer = sr.Recognizer()

            wav_filename = convert_mp3_to_wav(mp3_filename)

            with sr.AudioFile(wav_filename) as source:
                audio_text = recognizer.record(source)
                text = recognizer.recognize_google(audio_text, language=language)
                joined_text = text
        
        return jsonify({'text': joined_text})
    
@app.route('/api/translate_toar/<string:text>', methods=['GET'], endpoint='translate_to_ar')
def translate_to_ar(text):    
    translator = Translator() 
    arabic_translation = translator.translate(text,  src='auto', dest='ar').text
    return jsonify({'translated_txt': arabic_translation})

@app.route('/api/translate_totr/<string:text>', methods=['GET'], endpoint='translate_to_tr')
def translate_to_tr(text):  
    translator = Translator()    

    turkish_translation = translator.translate(text, src='auto', dest='tr').text
    return jsonify({'translated_txt': turkish_translation})

@app.route('/api/translate_toen/<string:text>', methods=['GET'], endpoint='translate_to_en')
def translate_to_en(text):  
    translator = Translator()    

    english_translation = translator.translate(text, src='auto', dest='en').text
    return jsonify({'translated_txt': english_translation})

@app.route('/api/translate_tohi/<string:text>', methods=['GET'], endpoint='translate_to_hi')
def translate_to_hi(text):  
    translator = Translator()    

    hindi_translation = translator.translate(text, src='auto', dest='hi').text
    return jsonify({'translated_txt': hindi_translation})

def add_newlines_every_n_words(input_string, n=10):
    words = input_string.split()
    output_string = ''
    for i, word in enumerate(words):
        if i > 0 and i % n == 0:
            output_string += '\n'
        output_string += word + ' '
    return output_string.strip()

@app.route('/api/findtopic/<string:text>', methods=['GET'])
def topic_finder(text):
    
    pipe = pipeline("text-classification", model="unitary/toxic-bert")
    pipe2 = pipeline("summarization", model="google/pegasus-xsum")

    topic = pipe(text)  
    topic = topic[0]['label']

    topic2 = pipe2(text, max_length = 20)
    topic2 = topic2[0]['summary_text']
    
    return jsonify({'topic': topic, 'topic2': topic2})

@app.route('/api/findSummary/<string:text>', methods=['GET'])
def summary_find(text):
    pipe = pipeline("summarization", model="google/pegasus-xsum")
    
    temp = add_newlines_every_n_words(text, 30)
    try: 
        summary = summarize(temp)
        print("len: ", len(summary))
    except:
        output = pipe(text) 
        summary = output[0]['summary_text']
             
    translator = Translator() 
    arabic_summary = translator.translate(summary, src='en', dest='ar').text
    turkish_summary = translator.translate(summary, src='en', dest='tr').text

    return jsonify({'summary_en': summary, 'summary_ar': arabic_summary, 'summary_tr': turkish_summary})


@app.route('/api/sentiment/<string:text>', methods=['GET'])
def sentiment(text):
    nltk.download('vader_lexicon')
    analyzer = SentimentIntensityAnalyzer()
    scores = analyzer.polarity_scores(text)
    positive_percent = round(scores['pos'] * 100, 2)
    negative_percent = round(scores['neg'] * 100, 2)

    total = positive_percent + negative_percent
    fin_pos = round((positive_percent/total)*100, 2)
    fin_neg = round((negative_percent/total)*100, 2)

    return jsonify({'positive': fin_pos, 'negative': fin_neg})

        
if __name__ == '__main__': 
    app.run(debug=False)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [13/Mar/2024 22:44:44] "POST /api/convert-mp3-to-text HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2024 22:44:46] "GET /api/translate_toen/American%20accent%20in%2010%20seconds%20is%20it%20is%20saying%20is%20he%20nice%20say%20easy%20nice%20easy%20nice%20easy%20nice%20nice HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2024 22:44:47] "GET /api/analyze-audio/American%20accent%20in%2010%20seconds%20is%20it%20is%20saying%20is%20he%20nice%20say%20easy%20nice%20easy%20nice%20easy%20nice%20nice HTTP/1.1" 200 -
config.json: 100%|██████████| 811/811 [00:00<?, ?B/s] 
model.safetensors:  19%|█▉        | 83.9M/438M [01:18<07:04, 833kB/s] Error while downloading from https://cdn-lfs.huggingface.co/unitary/toxic-bert/2c272885d24138df70bff1b3cd944a999bd6b41dad33209730aa8ba074f6ad09?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27model.safetensors%3B+filename%3D%22model.safetensors%22%3B&Expires=1710610928&Policy=eyJTdGF0ZW1lbnQi